In [33]:
import pandas as pd
import numpy as np
import copy, time
import math
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
df_train = pd.read_csv('./train_data.csv')
df_test = pd.read_csv('./test_features.csv')
print(df_train.shape)
columns=list(df_train.columns)
print(columns)
df_train.head()

(113, 22)
['name', 'bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'email_address', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'poi', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']


,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [34]:
train_Y=df_train['poi'] #目標欄位
name = df_test['name'] 
df_train = df_train.drop(['name','poi'] , axis=1) 
columns.remove('name')
columns.remove('poi')
df_test = df_test.drop(['name'] , axis=1)
df = pd.concat([df_train, df_test],sort=False) #資料一併做前處理
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [35]:
def na_num(df_data):
    num = (df_data.isnull().sum()/len(df)).sort_values(ascending=False)
    return num
na_num(df) #欄位缺失率

loan_advances                0.972603
director_fees                0.883562
restricted_stock_deferred    0.876712
deferral_payments            0.732877
deferred_income              0.664384
long_term_incentive          0.547945
bonus                        0.438356
from_messages                0.410959
from_poi_to_this_person      0.410959
from_this_person_to_poi      0.410959
shared_receipt_with_poi      0.410959
to_messages                  0.410959
other                        0.363014
expenses                     0.349315
salary                       0.349315
exercised_stock_options      0.301370
restricted_stock             0.246575
email_address                0.239726
total_payments               0.143836
total_stock_value            0.136986
dtype: float64

 'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock', 'restricted_stock_deferred',  'total_payments', 'total_stock_value' 

In [ ]:
#df['BAXTER JOHN C'].loc[] = 300000
#df['LAY KENNETH L'].loc = 

In [61]:
#缺失值比率過高，先捨去欄位
df = df.drop(['loan_advances'],axis=1)

In [55]:
#類別缺失值處理，改以是否有信箱
none_cols = ['email_address']

for col in none_cols:
    df[col]=df[col].isnull() 
    #df[col] = df[col].fillna("None")


In [56]:
#部分欄位缺失值填補0
zero_cols = ['long_term_incentive', 'deferred_income','deferral_payments','director_fees',\
            'from_messages','to_messages','from_poi_to_this_person', 'from_this_person_to_poi',\
            'shared_receipt_with_poi',\
             'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock',\
             'restricted_stock_deferred', 'total_payments', 'total_stock_value' ]

for col in zero_cols:
    df[col] = df[col].fillna(0)

In [84]:
#部分欄位缺失值填補眾數
mod_cols = ['salary','bonus']

for col in mod_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [82]:
#部分欄位缺失值填補平均數
med_cols = ['other','exercised_stock_options']

for col in med_cols:
    df[col] = df[col].fillna(df[col].median())

In [85]:
#確認缺失值填補是否完整
na_num(df) 

total_stock_value            0.0
from_poi_to_this_person      0.0
deferral_payments            0.0
deferred_income              0.0
director_fees                0.0
email_address                0.0
exercised_stock_options      0.0
expenses                     0.0
from_messages                0.0
from_this_person_to_poi      0.0
total_payments               0.0
long_term_incentive          0.0
other                        0.0
restricted_stock             0.0
restricted_stock_deferred    0.0
salary                       0.0
shared_receipt_with_poi      0.0
to_messages                  0.0
bonus                        0.0
dtype: float64

In [ ]:
#離群值處理
def remove_outlier(df):
    for k in df.columns[1:]:
        df_temp = df[k].copy()
        df_temp_iqr = iqr(df_temp)
        df_temp_std = np.std(df_temp)
        df_temp_med= np.median(df_temp)
        df_temp_q1=np.percentile(df_temp , 25)
        df_temp_q3=np.percentile(df_temp , 75)
        df_temp_lower = df_temp_q1-1.5*df_temp_iqr
        df_temp_upper = df_temp_q3+2*df_temp_iqr
        #df[k].fillna(df[k].median(),inplace=True)    
        df[k].loc[df_temp > df_temp_upper]=np.percentile(df_temp, q = 99.5)

In [90]:
# 將資料最大最小化
df = MinMaxScaler().fit_transform(df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
train_X = pd.DataFrame(df[:train_num])
test_X = pd.DataFrame(df[train_num:])

In [94]:
kf=KFold(n_splits = 5, shuffle = True, random_state = 42)
result=next(kf.split(train_X), None)
train=train_X.iloc[result[0]]
valid=train_X.iloc[result[1]]
df_train=train
df_train_Y=train_Y.iloc[result[0]]
df_valid=valid
df_valid_Y=train_Y.iloc[result[1]]

In [134]:
train_Y = train_Y.map(lambda x :int(x))

In [115]:
# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

lg = LogisticRegression()
gdbt = GradientBoostingRegressor(tol=0.1, subsample=0.37, n_estimators=200, #max_features=20, 
                                 max_depth=6, learning_rate=0.03)
rf = RandomForestRegressor(n_estimators=300, min_samples_split=9, min_samples_leaf=10,#max_features='', 
                           max_depth=8, bootstrap=False)

In [135]:
# 邏輯斯迴歸預測檔
from sklearn.feature_selection import f_regression
lg.fit(train_X, train_Y)
print(lg.coef_)
###########train###########
lg_predval = abs(lg.predict(df_valid))
print(lg_predval)
#linear_predval=linear_predval.map(lambda x:1 if x>0.5 else 0) 

###########tset###########
lg_pred = abs(lg.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': lg_pred})
#print(sub)
#sub.to_csv('house_linear.csv', index=False) 


[[ 0.08567073 -0.06142173 -0.91141283 -0.03740215 -1.31027664  0.25506886
   0.06366635 -0.37301394  0.32289687 -0.33277713  0.08987871  0.21023874
   0.12171751 -0.24875563  0.05050083  0.32871289 -0.21082022  0.27706519
   0.22547204]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [119]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
###########train###########
gdbt_predval = abs(gdbt.predict(df_valid))
print(gdbt_predval)

###########tset###########
gdbt_pred = abs(gdbt.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': np.expm1(gdbt_pred)})
print(sub)
#sub.to_csv('house_gdbt.csv', index=False)


[1.08549398e+00 8.89531704e-01 9.69741510e-01 8.42827168e-01
 9.09555630e-01 6.41472654e-02 1.09396997e-01 9.89029629e-02
 1.84365551e-02 4.23008245e-02 2.85171756e-03 1.79975786e-02
 1.67577146e-02 3.87162529e-02 9.27246301e-04 2.50338064e-03
 1.00794675e-01 1.52337313e-03 1.17238398e-01 7.43321643e-03
 5.34819738e-03 1.48849498e-02 7.89191114e-03]
                    name       poi
0       BELDEN TIMOTHY N  0.674348
1     BOWEN JR RAYMOND M  0.345130
2         HANNON KEVIN P  0.915730
3       DELAINEY DAVID W  0.076556
4       CAUSEY RICHARD A  0.515641
5       HICKERSON GARY J  0.478358
6         FREVERT MARK A  1.429339
7            CHAN RONNIE  0.031562
8   DONAHUE JR JEFFREY M  0.187091
9      REYNOLDS LAWRENCE  0.036192
10      HORTON STANLEY C  0.051781
11         LEWIS RICHARD  0.011608
12    SHERRICK JEFFREY B  0.029872
13        MEYER JEROME J  0.027039
14   DERRICK JR. JAMES V  0.841390
15              PIRO JIM  0.001756
16   DETMERING TIMOTHY J  0.027643
17     KISHKILL JO

In [120]:
# 隨機森林預測檔 
rf.fit(train_X, train_Y)
###########validation###########
rf_predval = abs(rf.predict(df_valid))
print(rf_predval)

###########test###########
rf_pred = abs(rf.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': np.expm1(rf_pred)})
print(sub)
#sub.to_csv('house_rf.csv', index=False)


[0.5        0.5        0.28571429 0.3        0.5        0.
 0.5        0.         0.         0.28571429 0.5        0.
 0.         0.         0.         0.         0.         0.
 0.28571429 0.         0.         0.         0.        ]
                    name       poi
0       BELDEN TIMOTHY N  0.648721
1     BOWEN JR RAYMOND M  0.648721
2         HANNON KEVIN P  0.648721
3       DELAINEY DAVID W  0.648721
4       CAUSEY RICHARD A  0.000000
5       HICKERSON GARY J  0.648721
6         FREVERT MARK A  0.648721
7            CHAN RONNIE  0.000000
8   DONAHUE JR JEFFREY M  0.330712
9      REYNOLDS LAWRENCE  0.349859
10      HORTON STANLEY C  0.000000
11         LEWIS RICHARD  0.000000
12    SHERRICK JEFFREY B  0.000000
13        MEYER JEROME J  0.000000
14   DERRICK JR. JAMES V  0.000000
15              PIRO JIM  0.000000
16   DETMERING TIMOTHY J  0.349859
17     KISHKILL JOSEPH G  0.330712
18        BAY FRANKLIN R  0.330712
19        PRENTICE JAMES  0.000000
20                 TOTAL  0.648

In [ ]:
# 混合泛化預測
blending_pred = lr_pred*0.2+ gdbt_pred*0.05+ rf_pred*0.75
sub = pd.DataFrame({'name': name, 'poi': np.expm1(y_pred)})
sub['poi'] = sub['poi'].map(lambda x:True if x>0.5 else False) 
sub.to_csv('EnronFraud.csv', index=False)